## Setup and Import Libraries

In [1]:
import os
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [4]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
llm = init_chat_model(model="groq:gemma2-9b-it")

## Document Loading and Splitting

In [5]:
loader = TextLoader(file_path="langchain_crewai_dataset.txt")
documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50
)

chunks = text_splitter.split_documents(documents=documents)

## Vector Store and Retriever

In [7]:
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "lambda_mult": 0.7}
)

## Query Decomposition Prompt and Chain

In [8]:
query_decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")

query_decomposition_prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.\n\nQuestion: "{question}"\n\nSub-questions:\n')

In [9]:
decomposition_chain = query_decomposition_prompt | llm | StrOutputParser()

In [10]:
query = "How does LangChain use memory and agents compared to CrewAI?"

decomposition_question = decomposition_chain.invoke(input={"question": query})

decomposition_question

"Here are some sub-questions that break down the original complex question:\n\n1. **What types of memory mechanisms does LangChain utilize?**  (This focuses on understanding LangChain's memory capabilities)\n2. **How does LangChain integrate agents into its framework?** (This explores LangChain's approach to agent-based functionality)\n3. **What are the memory and agent capabilities of CrewAI?** (This establishes a baseline for comparison) \n4. **What are the key differences in memory management and agent implementation between LangChain and CrewAI?** (This prompts a direct comparison) \n\n\nBy answering these sub-questions, you can gain a comprehensive understanding of how LangChain and CrewAI handle memory and agents, allowing for a more informed comparison. \n\n"

## QA Prompt and Chain

In [11]:
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")

qa_prompt

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nUse the context below to answer the question.\n\nContext:\n{context}\n\nQuestion: {input}\n')

In [12]:
qa_chain = create_stuff_documents_chain(
    llm=llm, 
    prompt=qa_prompt
)

qa_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nUse the context below to answer the question.\n\nContext:\n{context}\n\nQuestion: {input}\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002341CAF8D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002341D0E5A60>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

## RAG Pipeline

In [13]:
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke(input={"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for sub_question in sub_questions:
        docs = retriever.invoke(sub_question)
        result = qa_chain.invoke(input={"input": sub_question, "context": docs})
        results.append(f"Q: {sub_question}\nA: {result}")
    
    return "\n\n".join(results)

## Query

In [15]:
query = "How does LangChain use memory and agents compared to CrewAI?"

final_answer = full_query_decomposition_rag_pipeline(user_query=query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are some sub-questions that break down the complex query:
A: Please provide the complex query you would like to break down into sub-questions.  

For example, you could ask: 

"Here is a complex query: **How can I use CrewAI to analyze a legal contract for potential risks?** 

Here are some sub-questions that break down the complex query:" 


I can then help you formulate relevant sub-questions based on the provided context and your query. 



Q: **What types of memory mechanisms does LangChain utilize?**
A: According to the context, LangChain utilizes memory modules such as:

* **ConversationBufferMemory**
* **ConversationSummaryMemory** 


These modules help the LLM remember past conversation turns or summarize long interactions. 


Q: **How do LangChain agents leverage memory for task completion?**
A: LangChain agents use **context-aware memory** to aid in task completion. This means they can remember and utilize information from previous steps in a sequence